In [7]:
import pandas as pd
from sqlalchemy import create_engine
from config import username, password

In [8]:
billionaire_df = pd.read_csv("../data/billionaire.csv")
billionaire_df.head()

,Name,NetWorth,Country,Source,Rank
0,Jeff Bezos,$177 B,United States,Amazon,1
1,Elon Musk,$151 B,United States,"Tesla, SpaceX",2
2,Bernard Arnault & family,$150 B,France,LVMH,3
3,Bill Gates,$124 B,United States,Microsoft,4
4,Mark Zuckerberg,$97 B,United States,Facebook,5


In [9]:
coord_df = pd.read_csv("../data/country_lat_lon.csv")
coord_df.head()

,country_code,latitude,longitude,country,usa_state_code,usa_state_latitude,usa_state_longitude,usa_state
0,AD,42.546245,1.601554,Andorra,AK,63.588753,-154.493062,Alaska
1,AE,23.424076,53.847818,United Arab Emirates,AL,32.318231,-86.902298,Alabama
2,AF,33.939110,67.709953,Afghanistan,AR,35.201050,-91.831833,Arkansas
3,AG,17.060816,-61.796428,Antigua and Barbuda,AZ,34.048928,-111.093731,Arizona
4,AI,18.220554,-63.068615,Anguilla,CA,36.778261,-119.417932,California


In [10]:
merged_df = pd.merge(billionaire_df, coord_df, left_on="Country", right_on="country")
merged_df.head()

,Name,NetWorth,Country,Source,Rank,country_code,latitude,longitude,country,usa_state_code,usa_state_latitude,usa_state_longitude,usa_state
0,Jeff Bezos,$177 B,United States,Amazon,1,US,37.09024,-95.712891,United States,NaN,NaN,NaN,NaN
1,Elon Musk,$151 B,United States,"Tesla, SpaceX",2,US,37.09024,-95.712891,United States,NaN,NaN,NaN,NaN
2,Bill Gates,$124 B,United States,Microsoft,4,US,37.09024,-95.712891,United States,NaN,NaN,NaN,NaN
3,Mark Zuckerberg,$97 B,United States,Facebook,5,US,37.09024,-95.712891,United States,NaN,NaN,NaN,NaN
4,Warren Buffett,$96 B,United States,Berkshire Hathaway,6,US,37.09024,-95.712891,United States,NaN,NaN,NaN,NaN


In [44]:
final_df = merged_df[["Name", "NetWorth", "Country", "Source", "Rank", "latitude", "longitude"]].copy()

final_df = final_df.rename(columns={"Name": "name",
                                    "NetWorth": "networth",
                                    "Country": "country",
                                    "Source": "source",
                                    "Rank": "rank"})

final_df["networth"] = final_df["networth"].map(lambda x: x.lstrip("$").rstrip(" B")).astype(float)

final_df.dtypes

name          object
networth     float64
country       object
source        object
rank           int64
latitude     float64
longitude    float64
dtype: object

In [15]:
# final_df.to_csv("../data/merged_data.csv", index=False)

In [45]:
final_df["id"] = final_df.index
final_df.set_index("id", inplace=True)
final_df.head()

,name,networth,country,source,rank,latitude,longitude
id,,,,,,,
0,Jeff Bezos,177.0,United States,Amazon,1,37.09024,-95.712891
1,Elon Musk,151.0,United States,"Tesla, SpaceX",2,37.09024,-95.712891
2,Bill Gates,124.0,United States,Microsoft,4,37.09024,-95.712891
3,Mark Zuckerberg,97.0,United States,Facebook,5,37.09024,-95.712891
4,Warren Buffett,96.0,United States,Berkshire Hathaway,6,37.09024,-95.712891


In [46]:
# engine = create_engine("sqlite:///../data/billionaires.sqlite")
connection_string = f"{username}:{password}@localhost:5432/billionaires_db"
engine = create_engine(f"postgresql://{connection_string}")

In [47]:
final_df.to_sql(name="billionaires", con=engine, if_exists="append", index=True)

In [48]:
pd.read_sql_query('select * from billionaires', con=engine).head()

,id,name,networth,country,source,rank,latitude,longitude
0,0,Jeff Bezos,177.0,United States,Amazon,1,37.09024,-95.712891
1,1,Elon Musk,151.0,United States,"Tesla, SpaceX",2,37.09024,-95.712891
2,2,Bill Gates,124.0,United States,Microsoft,4,37.09024,-95.712891
3,3,Mark Zuckerberg,97.0,United States,Facebook,5,37.09024,-95.712891
4,4,Warren Buffett,96.0,United States,Berkshire Hathaway,6,37.09024,-95.712891
